In [0]:
%pip install openai

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
spark.conf.set(
    "fs.azure.account.key.mystore.dfs.core.windows.net",
    "XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX"
)

In [0]:
from pyspark.sql.functions import expr
df = spark.read.csv("abfss://rawdata@mystore1984.dfs.core.windows.net/agnatic data.csv", inferSchema= True, header= True)\
        .withColumn('Date', expr('to_date(Date,"dd-MM-yyyy")'))\
        .withColumn('Month', expr('date_format(Date, "yyyy-MM")'))
df.show(5)

+----------+--------------------+-------+-------------+----+---+------+-----------+-------+
|      Date|              Status|  Style|     Category|Size|Qty|Amount|  ShipState|  Month|
+----------+--------------------+-------+-------------+----+---+------+-----------+-------+
|2022-12-04|             Shipped|  J0080|          Top|  XS|  1| 531.0|    Gujarat|2022-12|
|2022-12-04|             Shipped|  J0122|          Top| 3XL|  1| 329.0|MAHARASHTRA|2022-12|
|2022-12-04|Shipped - Deliver...|  J0400|Western Dress|   M|  1| 859.0|    HARYANA|2022-12|
|2022-12-04|Shipped - Deliver...|JNE3801|        kurta|   L|  1| 725.0|     ODISHA|2022-12|
|2022-12-04|             Shipped| SET401|          Set|   M|  1|1186.0|  TELANGANA|2022-12|
+----------+--------------------+-------+-------------+----+---+------+-----------+-------+
only showing top 5 rows



In [0]:
OPENAI_ENDPOINT = "https://XXXXXXXXXXXX.openai.azure.com/"
OPENAI_KEY = "XXXXXXXXXXXXXXXXXXXXXXX"
from openai import AzureOpenAI

EMBEDDING_MODEL = "agnetic_model"

client = AzureOpenAI(
    api_key=OPENAI_KEY,
    azure_endpoint=OPENAI_ENDPOINT,
    api_version="2024-12-01-preview"
)

In [0]:
def run_python(code: str):
    local_vars = {}
    exec(code, globals(), local_vars)

    if "result" in local_vars:
        if hasattr(local_vars["result"], "toPandas"):
            return local_vars["result"].toPandas().to_dict()
        return local_vars["result"]

    return "Code executed successfully"


In [0]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "run_python",
            "description": "Execute Python or Spark code in Databricks",
            "parameters": {
                "type": "object",
                "properties": {
                    "code": {
                        "type": "string",
                        "description": "Executable Python/Spark code"
                    }
                },
                "required": ["code"]
            }
        }
    }
]


In [0]:
system_prompt = """
You are a senior data analytics agent running inside Databricks.

Rules:
- Use python for data access for already loaded dataframe df
- Assign final output to variable named `result`
- Do not print anything
- If asked to store results, write to Hive Metastore 
- If calculation is required, generate executable Python code
- You may store results to SQL tables if asked
"""


In [0]:
user_question = """
Calculate the total Amount for each Category where Month is 2022-12.
Select the top 3 Categories based on total Amount.
Store the result in Hive Metastore table
hive_metastore.default.sales_dec_2022
"""

In [0]:
response = client.chat.completions.create(
    model="agnetic_model",   
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_question}
    ],
    tools=tools,
    tool_choice="auto"
)


In [0]:
message = response.choices[0].message
import json
import pandas as pd

if message.tool_calls:
    for tool_call in message.tool_calls:
        if tool_call.function.name == "run_python":
            args = json.loads(tool_call.function.arguments)
            tool_result = run_python(args["code"])

            # Send execution result back to OpenAI
            response = client.chat.completions.create(
                model="agnetic_model",
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_question},
                    message,
                    {
                        "role": "tool",
                        "tool_call_id": tool_call.id,
                        "content": json.dumps(tool_result)
                    }
                ]
            )


In [0]:
final_answer = response.choices[0].message.content
final_answer


Out[10]: 'The top 3 categories based on total amount for December 2022 have been calculated and stored in the Hive Metastore table `hive_metastore.default.sales_dec_2022`.'

In [0]:
display(tool_result)


'Code executed successfully'

In [0]:
%sql
use catalog `hive_metastore`; select * from `default`.`sales_dec_2022` limit 100;

Category,Total_Amount
Set,1240030.44
kurta,650528.2999999999
Western Dress,388782.29


Databricks visualization. Run in Databricks to view.